In [7]:
import pandas as pd
import numpy as np
import scipy
from nltk.corpus import stopwords
import re
import os

In [8]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)

tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case


tranzact_data = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data = tranzact_data.reset_index(drop=True)


In [9]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap

## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())


## Removing duplicate words from individual features

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
                                                                                key=x.split().index))]).lower())

## Removing only single word features :

tranzact_data = tranzact_data[tranzact_data['features'].str.contains(' ')]
tranzact_data = tranzact_data.reset_index(drop=True)

print(len(tranzact_data))

27044


In [10]:
for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        tranzact_data['Supplier'][i] = np.nan
    
        
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))

27044


In [5]:
## To check no. of Suppliers & no. of unique featuresin the data base :

# print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data.features.value_counts()))
# print('Tranzact_data length = ', len(tranzact_data))
# tranzact_data.tail()

In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [11]:
## Removing duplicate features + Supplier combinations :

for i in range (len(tranzact_data)):
    
    idx_feature = []
    idx_Supplier = []
    idx_common = []

    idx_feature = tranzact_data.index[tranzact_data['features'] == tranzact_data['features'][i]].tolist()
    idx_Supplier = tranzact_data.index[tranzact_data['Supplier'] == tranzact_data['Supplier'][i]].tolist()
    idx_common = list(set(idx_feature).intersection(idx_Supplier))
    
    if len(idx_common) > 1:
        
        tranzact_data['Supplier'][idx_common[1:]] = np.nan
        
tranzact_data_final = tranzact_data.dropna()
tranzact_data_final = tranzact_data_final.reset_index(drop=True)                           

In [7]:
# tranzact_data_final.tail()

In [8]:
## To cross check no. of Suppliers & no. of unique features in the data base :

# print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
# print('Total no. of entrys = ', len(tranzact_data_final))
# tranzact_data_final.tail()

In [12]:
# Creating seacrh engine file  and digitising original features :

from sklearn.feature_extraction.text import TfidfVectorizer

col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col]


tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)

## Digitising :

my_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))
digital_features = my_vectorizer.fit(tranzact_data_search['features'])
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()

In [13]:
## Input Text data :

text_data = ['lock nut']

In [14]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in text_data]

text_data1 = pd.Series(text_data_lower_case)

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
                                                                                key=x.split().index))]).lower())
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []

merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()


test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

## Finding cosine similarity :

c_d = []

c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)


In [15]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]

for i in max_c_s_index:
    
    result.append(np.array([c_d[i], tranzact_data_final['Item'][i], tranzact_data_search['Supplier'][i]]))

result_frame = pd.DataFrame(result, columns=['c_index', 'Item', 'Supplier'])

result_frame.c_index = result_frame.c_index.astype('float64')

result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['Supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['Supplier'][i])
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

MK Engineering Works
Caliber Enterprises
VAKRATUNDA
CESARE BONETTI INDIA PVT. LTD.
Shah Brothers


,c_index,Item,Supplier
0,1.000,lock nut,MK Engineering Works
1,1.000,lock nut,Caliber Enterprises
2,1.000,lock nut,VAKRATUNDA
3,1.000,lock nut,CESARE BONETTI INDIA PVT. LTD.
4,1.000,lock nut,Shah Brothers
5,0.600,lock nut m27x3p,Mahavir Enterprises
6,0.556,lock nut m20x2.5p,Waaree Industries Pvt.Ltd.
7,0.541,lock nut m12x1.25p,DAMODAR ENTERPRISES
